# Strade

In [32]:
import fiona
import geopandas as gpd
from fiona.transform import transform_geom

In [ ]:
#----- PATH Confini ----#
PATH_CONFINI_RAW = "../Data/Raw/strade_urbane/confini-milano_raw.geojson"

#----- PATH Strade -----#
PATH_INTERSEZIONI_RAW = "../Data/Raw/strade_urbane/REGIONE_LOMBARDIA/Intersezioni_Stradali.shp"
PATH_INTERSEZIONI_STAGIN = "../Data/Staging/strade_urbane/intersezioni_stradali.geojson"
PATH_INTERSEZIONI_CLEAN = "../Data/Clean/strade_urbane/intersezioni_stradali.geojson"

PATH_STRADE_PRINCIPALI_RAW = "../Data/Raw/strade_urbane/REGIONE_LOMBARDIA/Strade_Principali.shp"
PATH_STRADE_PRINCIPALI_STAGIN ="../Data/Staging/strade_urbane/Strade_Principali.geojson"
PATH_STRADE_PRINCIPALI_CLEAN = "../Data/Clean/strade_urbane/Strade_Principali.geojson"

PATH_STRADE_SECONDARIE_RAW = "../Data/Raw/strade_urbane/REGIONE_LOMBARDIA/Strade_Secondarie.shp"
PATH_STRADE_SECONDARIE_STAGIN ="../Data/Staging/strade_urbane/Strade_Secondarie.geojson"
PATH_STRADE_SECONDARIE_CLEAN = "../Data/Clean/strade_urbane/Strade_Secondarie.geojson"

## ETL

Conversione e salvataggio in Staging in formato geojson

In [34]:
#intersezioni

src_crs = 'EPSG:32632'
dst_crs = 'EPSG:4326'

with fiona.open(PATH_INTERSEZIONI_RAW, 'r') as src:
    schema = src.schema.copy()
    crs_dst = fiona.crs.from_string(dst_crs)

    with fiona.open(PATH_INTERSEZIONI_STAGIN, 'w', driver='GeoJSON', crs=crs_dst, schema=schema) as dst:
        for feature in src:
            geom_transformed = transform_geom(src_crs, dst_crs, feature['geometry'])
            feature['geometry'] = geom_transformed
            dst.write(feature)

print(" GeoJSON convertito e salvato correttamente.")

C:\Users\apera\AppData\Local\Temp\ipykernel_23168\3841102503.py:13: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  feature['geometry'] = geom_transformed


 GeoJSON convertito e salvato correttamente.


In [35]:

#strade principali

src_crs1 = 'EPSG:32632'
dst_crs1 = 'EPSG:4326'

with fiona.open(PATH_STRADE_PRINCIPALI_RAW, 'r') as src1:
    schema = src1.schema.copy()
    crs_dst1 = fiona.crs.from_string(dst_crs1)

    with fiona.open(PATH_STRADE_PRINCIPALI_STAGIN, 'w', driver='GeoJSON', crs=crs_dst1, schema=schema) as dst1:
        for feature in src1:
            geom_transformed1 = transform_geom(src_crs1, dst_crs1, feature['geometry'])
            feature['geometry'] = geom_transformed1
            dst1.write(feature)

print(" GeoJSON convertito e salvato correttamente.")

C:\Users\apera\AppData\Local\Temp\ipykernel_23168\3053756607.py:13: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  feature['geometry'] = geom_transformed1


 GeoJSON convertito e salvato correttamente.


In [36]:
# ETL strade secodarie

gdf_strade_secondarie = gpd.read_file(PATH_STRADE_SECONDARIE_RAW)

gdf_strade_secondarie = gdf_strade_secodarie.to_crs(4326)

gdf_strade_secondarie.to_file("../Data/Staging/strade_urbane/Strade_Secondarie.geojson", driver = "GeoJSON")

Filtraggio sui confini di Milano

In [37]:
#inserimento confini su intersezioni

gdf_confini = gpd.read_file(PATH_CONFINI_RAW)
gdf_intersezioni = gpd.read_file(PATH_INTERSEZIONI_STAGIN)

dati_filtrati = gpd.sjoin(gdf_intersezioni, gdf_confini, how="inner", predicate="intersects")

In [38]:
#inserimento confini su strade principali

gdf_confini = gpd.read_file(PATH_CONFINI_RAW)
gdf_strade_principali = gpd.read_file(PATH_STRADE_PRINCIPALI_STAGIN)

dati_filtrati_str_principali = gpd.sjoin(gdf_strade_principali, gdf_confini, how="inner", predicate="intersects")

In [39]:
#inserimento confini su strade secondarie

gdf_confini = gpd.read_file(PATH_CONFINI_RAW)
gdf_strade_secondarie = gpd.read_file(PATH_STRADE_PRINCIPALI_STAGIN)

dati_filtrati_str_secondarie = gpd.sjoin(gdf_strade_secondarie, gdf_confini, how="inner", predicate="intersects")

Salvataggio in Clean

In [41]:
#dati_filtrati.to_file(PATH_INTERSEZIONI_CLEAN, driver="GeoJSON")
#dati_filtrati_str_principali.to_file(PATH_STRADE_PRINCIPALI_CLEAN, driver="GeoJSON")
dati_filtrati_str_secondarie.to_file(PATH_STRADE_SECONDARIE_CLEAN, driver="GeoJSON")
